## Geomorphometry II: Spatial and Temporal Terrain Analysis

Resources: [
GRASS GIS overview and manual](https://grass.osgeo.org/grass76/manuals/index.html),
[GRASSbook](http://www.grassbook.org/).


Download Mapset and color tables:

*  Download [
NagsHead time series](http://fatra.cnr.ncsu.edu/geospatial-modeling-course/data/NagsHead_series.zip) and copy it into your nc_spm_08_grass7 Location
(it should be the same level directory as PERMANENT).
Do not let your unzipping program create additional level directory with the same name!
If you are not sure about GRASS GIS Database structure read about it in
[the manual](https://grass.osgeo.org/grass76/manuals/grass_database.html).
* Custom color table for time series standard deviations map [stddev_color.txt](data/stddev_color.txt)



### Start GRASS GIS
Start GRASS - click on GRASS icon or type

In [ ]:
# This is a quick introduction into Jupyter Notebook.
# Python code can be executed like this:
a = 6
b = 7
c = a * b
print("Answer is", c)
# Python code can be mixed with command line code (Bash).
# It is enough just to prefix the command line with an exclamation mark:
!echo "Answer is $c"
# Use Shift+Enter to execute this cell. The result is below.

In [ ]:
import os
import sys
import subprocess
from IPython.display import Image

# create GRASS GIS runtime environment
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ['GISBASE'] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# do GRASS GIS imports
import grass.script as gs
import grass.script.setup as gsetup

# set GRASS GIS session data
rcfile = gsetup.init(gisbase, "/home/jovyan/grassdata", "nc_spm_08_grass7", "user1")

In [ ]:
# default font displays
os.environ['GRASS_FONT'] = 'sans'
# overwrite existing maps
os.environ['GRASS_OVERWRITE'] = '1'
gs.set_raise_on_error(True)
gs.set_capture_stderr(True)

In [ ]:
# set display modules to render into a file (named map.png by default)
os.environ['GRASS_RENDER_IMMEDIATE'] = 'cairo'
os.environ['GRASS_RENDER_FILE_READ'] = 'TRUE'
os.environ['GRASS_LEGEND_FILE'] = 'legend.txt'

In startup pannel set GRASS GIS Database Directory to path to datasets,
for example on MS Windows, `C:\Users\myname\grassdata`.
For GRASS Location select nc_spm_08_grass7 (North Carolina, State Plane, meters) and
for GRASS Mapset create a new mapset (called e.g. HW_terrain_analysis).
Click Start GRASS session.


Change working directory:
_Settings_ > _GRASS working environment_ > _Change working directory_ > select/create any directory
or type `cd` (stands for change directory) into the GUI
_Console_ and hit Enter:

In [ ]:
# a proper directory is already set, download files
import urllib.request
urllib.request.urlretrieve("http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/data/stddev_color.txt", "stddev_color.txt")

Download all text files (see above)
to the selected directory. Now you can use the commands from the assignment requiring the text file
without the need to specify the full path to the file.


### Compute basic topographic parameters: slope and aspect

In [ ]:
gs.parse_command('g.region', raster="elevation", flags='pg')
gs.run_command('r.slope.aspect', elevation="elevation", slope="myslope", aspect="myaspect")

Display resulting maps with legend using GUI.

In [ ]:
gs.run_command('d.rast', map="myslope")
gs.run_command('d.legend', raster="myslope", at="2,40,2,6")
Image(filename="map.png")
gs.run_command('d.rast', map="myaspect")
gs.run_command('d.legend', raster="myaspect", at="2,40,2,6")
Image(filename="map.png")

Show impact of integer values in meters on slope and aspect pattern.
Compute integer DEM and derive its slope and aspect.
Use GUI to display the histogram: in _Map Display_ > _Analyze_ > _Create histogram_:

In [ ]:
gs.mapcalc("elev_int = int(elevation)")
gs.run_command('r.slope.aspect', elevation="elev_int", aspect="aspect_int_10m", slope="slope_int_10m")
gs.run_command('d.erase')
gs.run_command('d.histogram', map="slope_int_10m")
gs.run_command('d.histogram', map="myslope")
gs.run_command('d.histogram', map="aspect_int_10m")
gs.run_command('d.histogram', map="myaspect")
gs.run_command('d.erase')
gs.run_command('d.rast', map="myslope")
Image(filename="map.png")

Zoom into NW area of the current region (relatively flat area near large interchange).
Can you explain the difference in slope maps derived from integer (m vertical resolution)
and floating point (mm vertical resolution) DEMs?

In [ ]:
gs.run_command('d.rast', map="slope_int_10m")
Image(filename="map.png")

### Compute slope along road
First set the region to the extent of the bus route #11 and to 10m resolution.
Then convert the vector line of the route to raster using the direction of the route.

In [ ]:
gs.parse_command('g.region', vect="busroute11", align="elevation", res="10", flags='pg')
gs.run_command('v.to.rast', input="busroute11", type="line", output="busroute11_dir", use="dir")
gs.run_command('r.colors', map="busroute11_dir", color="aspect")
gs.run_command('d.rast', map="busroute11_dir")
gs.run_command('d.legend', raster="busroute11_dir")
Image(filename="map.png")

Compute the steepest slope of the topography along the route by assigning
the values of slope derived from a DEM in the first part of this assignment
to the grid cells along the route.

In [ ]:
gs.mapcalc("route_slope = if(busroute11_dir, myslope)")

Then compute the slope in the direction of the route using the difference between aspect of the topography
and the route direction angles.

In [ ]:
gs.mapcalc("route_slope_dir = abs(atan(tan(myslope) * cos(myaspect - busroute11_dir)))")
gs.run_command('r.colors', map="route_slope,route_slope_dir", color="slope")

Display the results along with the elevation contours and compute univariate statistics.
Comment on the difference of the two results.

In [ ]:
gs.run_command('r.contour', input="elevation", output="contours", step="2")
gs.run_command('d.vect', map="contours")
gs.run_command('d.rast', map="route_slope")
gs.run_command('d.legend', raster="route_slope")
Image(filename="map.png")
gs.run_command('d.rast', map="route_slope_dir")
Image(filename="map.png")
gs.parse_command('r.univar', map="route_slope", flags='g')
gs.parse_command('r.univar', map="route_slope_dir", flags='g')
Image(filename="map.png")

### Curvatures



Compute slope, aspect and curvatures simultaneously with interpolation.
You can do the examples below for the bare earth data only (first example),
multiple return example is optional (if you are curious how it differs from BE).

In [ ]:
gs.parse_command('g.region', region="rural_1m", flags='pg')
gs.run_command('v.surf.rst', input="elev_lid792_bepts", elevation="elev_lid_1m", aspect="asp_lid_1m", pcurvature="pc_lid_1m", tcurvature="tc_lid_1m", npmin="120", segmax="25")
gs.run_command('v.surf.rst', input="elev_lidrural_mrpts", elevation="elev_lidmr_1m", aspect="asp_lidmr_1m", pcurvature="pc_lidmr_1m", tcurvature="tc_lidmr_1m", npmin="120", segmax="25", tension="300", smooth="1.")

Display the results as 2D images or in 3D view.
For 3D view, switch off everything except for elevation surface that you want to view.
Drape topographic parameters raster maps over DEMs as color.

In [ ]:
gs.run_command('d.erase')
gs.run_command('d.rast', map="elev_lid_1m")
gs.run_command('d.rast', map="pc_lid_1m")
Image(filename="map.png")
gs.run_command('d.rast', map="elev_lidmr_1m")
gs.run_command('d.rast', map="pc_lidmr_1m")
Image(filename="map.png")

The curvature maps reflect survey pattern rather than topographic features.
So we lower the tension and increase the smoothing.
You can use multiple displays to compare the results side-by-side.

In [ ]:
gs.parse_command('g.region', region="rural_1m", flags='pg')
gs.run_command('v.surf.rst', input="elev_lid792_bepts", elevation="elev_lidt15_1m", aspect="asp_lidt15_1m", pcurvature="pc_lidt15_1m", tcurvature="tc_lidt15_1m", npmin="120", segmax="25", tension="15", smooth="1.")
gs.run_command('v.surf.rst', input="elev_lidrural_mrpts", elevation="elev_lidmrt15_1m", aspect="asp_lidmrt15_1m", pcurvature="pc_lidmrt15_1m", tcurvature="tc_lidmrt15_1m", npmin="120", segmax="25", tension="15", smooth="1.")
gs.run_command('d.erase')
gs.run_command('d.rast', map="elev_lidt15_1m")
gs.run_command('d.rast', map="pc_lidt15_1m")
Image(filename="map.png")
gs.run_command('d.rast', map="tc_lidt15_1m")
gs.run_command('d.rast', map="elev_lidmrt15_1m")
gs.run_command('d.rast', map="pc_lidmrt15_1m")
Image(filename="map.png")

### Landforms


Extract landforms at different levels of detail by adjusting the size of moving window.
Set rural subregion at 1m resolution,
compute landforms using 9m and 45m neighborhood: read the manual to learn more.
Explain types of landforms and the role of the neighborhood size.

In [ ]:
gs.parse_command('g.region', region="rural_1m", flags='pg')
gs.run_command('r.param.scale', input="elev_lid792_1m", output="feature9c_1m", size="9", method="feature")
gs.run_command('r.param.scale', input="elev_lid792_1m", output="feature45c_1m", size="45", method="feature")

Display with legend, save images for the report.
Optionally display the feature maps draped over elev_lid792_1m as color.

In [ ]:
gs.run_command('d.rast', map="feature9c_1m")
gs.run_command('d.legend', raster="feature9c_1m", at="2,20,2,6")
gs.run_command('d.rast', map="feature45c_1m")
gs.run_command('d.legend', raster="feature45c_1m", at="2,20,2,6")
gs.run_command('d.vect', map="elev_lid792_cont1m", color="brown")
Image(filename="map.png")

### Raster time series analysis

For this exercise we will use NagsHead_series Mapset you downloaded.
You have to first make the mapset accessible.
In GUI: menu _ Settings_ -> _GRASS working environment_ -> _Mapset access_
or by using a command:

In [ ]:
gs.run_command('g.mapsets', operation="add", mapset="NagsHead_series")

If you don't see the mapset, make sure you downloaded it and unzipped it correctly.


Run the series analysis and explain the results:
Which maps are core and envelope? 
Which landforms have high standard deviation and what does it mean?

In [ ]:
gs.parse_command('g.region', raster="NH_2008_1m", flags='pg')
gs.run_command('d.erase')
gs.run_command('d.rast', map="NH_2008_1m")
gs.run_command('r.series', input="NH_1999_1m,NH_2001_1m,NH_2004_1m,NH_2005_1m,NH_2007_1m,NH_2008_1m", out="NH_9908_min", method="minimum")
gs.run_command('r.series', input="NH_1999_1m,NH_2001_1m,NH_2004_1m,NH_2005_1m,NH_2007_1m,NH_2008_1m", out="NH_9908_max", method="maximum")
gs.run_command('r.series', input="NH_1999_1m,NH_2001_1m,NH_2004_1m,NH_2005_1m,NH_2007_1m,NH_2008_1m", out="NH_9908_mintime", method="min_raster")
gs.run_command('r.series', input="NH_1999_1m,NH_2001_1m,NH_2004_1m,NH_2005_1m,NH_2007_1m,NH_2008_1m", out="NH_9908_maxtime", method="max_raster")
gs.run_command('r.series', input="NH_1999_1m,NH_2001_1m,NH_2004_1m,NH_2005_1m,NH_2007_1m,NH_2008_1m", out="NH_9908_range", method="range")
gs.run_command('r.series', input="NH_1999_1m,NH_2001_1m,NH_2004_1m,NH_2005_1m,NH_2007_1m,NH_2008_1m", out="NH_9908_avg", method="average")
gs.run_command('r.series', input="NH_1999_1m,NH_2001_1m,NH_2004_1m,NH_2005_1m,NH_2007_1m,NH_2008_1m", out="NH_9908_stddev", method="stddev")
gs.run_command('r.colors', map="NH_9908_stddev", rules="stddev_color.txt")
gs.run_command('d.rast', map="NH_9908_stddev")
gs.run_command('d.rast', map="NH_9908_range")
Image(filename="map.png")

Use cutting plane in 3D view to show the core and envelope.
Add constant elevation plane at -1m for reference,
set zexag somewhere 3-5 (the default is too high).
Assign surfaces constant color, use top or bottom surface for crossection color.
When using top for color, lower the light source to make
top surface dark and highlight the crossection.




### Optional: Cut and fill and volume



Compute cut and fill for 4m deep excavation to build a facility.
First, set the region and display facility on top of orthophoto.

In [ ]:
gs.parse_command('g.region', region="rural_1m", flags='pg')
gs.run_command('d.erase')
gs.run_command('d.rast', map="ortho_2001_t792_1m")
gs.run_command('d.rast', map="facility")
Image(filename="map.png")

Then set (raster) mask to the facility map and find minimum elevation
within the facility:

In [ ]:
gs.run_command('r.mask', raster="facility")
gs.parse_command('r.univar', map="elevation", flags='g')

Minimum which you obtain should be 123.521m.
Bottom of 4m excavation will be then

```
123.52 - 4 = 119.52
```

Use raster algebra to create the excavation:

In [ ]:
gs.mapcalc("excavation = elevation - 119.52")
gs.parse_command('r.univar', map="excavation", flags='g')
gs.run_command('d.rast', map="excavation")
Image(filename="map.png")

Minimum you get should be 4.00057 and maximum 9.50554.
Note that the excavation is limited by the mask we set earlier,
so we can now do global operation to compute the volume
which applies just the the facility.

In [ ]:
gs.run_command('r.volume', map="excavation")

Now remove mask. This is important so that
your future work is not affected.

In [ ]:
gs.run_command('r.mask', flags='r')

In [ ]:
# end the GRASS session
os.remove(rcfile)